In [1]:
! pip install pandas scikit-learn nltk gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 13.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 12.2 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 7.7 MB/s eta 0:00:00m eta 0:00:010:00:01m
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 5.9 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 8.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 KB 7.7 MB/s eta 0:00:009.9 MB/s eta 0:00:01
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached click-8.2.1-py3-none-any.whl (102 kB)
     ━━━━

In [1]:
import pandas as pd 
data = pd.read_csv('input/data.csv',header=None)
data.head()

,0,1,2
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...


In [2]:
# polarity - 1 for negative and 2 for positive
# title - review heading
# text - review body
data.columns = ['polarity', 'title', 'text']

In [3]:
data['title'][3]

'works fine, but Maha Energy is better'

In [4]:
data['text'][3]

"Check out Maha Energy's website. Their Powerex MH-C204F charger works in 100 minutes for rapid charge, with option for slower charge (better for batteries). And they have 2200 mAh batteries."

In [5]:
data = data['text']
data.head()

0    My lovely Pat has one of the GREAT voices of h...
1    Despite the fact that I have only played a sma...
2    I bought this charger in Jul 2003 and it worke...
3    Check out Maha Energy's website. Their Powerex...
4    Reviewed quite a bit of the combo players and ...
Name: text, dtype: object

In [6]:
data.shape

(400000,)

In [7]:
data = data[:5000]

In [8]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

stopwords_en = stopwords.words('english')
lemmatizer   = WordNetLemmatizer()
def preprocess(text):
    tokens = word_tokenize(text.lower())
    return [lemmatizer.lemmatize(token) for token in tokens if token not in stopwords_en and token.isalpha()]


[nltk_data] Downloading package punkt_tab to /home/eps/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/eps/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/eps/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
preprocess('HI i am eps')

['hi', 'eps']

In [10]:
processed_texts = data.apply(preprocess)

In [11]:
processed_texts[:10]

0    [lovely, pat, one, great, voice, generation, l...
1    [despite, fact, played, small, portion, game, ...
2    [bought, charger, jul, worked, ok, design, nic...
3    [check, maha, energy, website, powerex, charge...
4    [reviewed, quite, bit, combo, player, hesitant...
5    [also, began, incorrect, disc, problem, read, ...
6    [love, style, couple, year, dvd, giving, probl...
7    [scroll, dvd, menu, set, vertically, triangle,...
8    [exotic, tale, orient, dr, shen, fu, weird, ta...
9    [firstly, enjoyed, format, tone, book, author,...
Name: text, dtype: object

In [12]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_texts)
corpus = [dictionary.doc2bow(text) for text in processed_texts]


In [13]:
len(corpus)

5000

In [14]:
len(corpus[0])

50

In [15]:
from gensim.models.ldamodel import LdaModel

lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=5)

In [16]:
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.050*"book" + 0.019*"read" + 0.011*"story" + 0.009*"one" + 0.007*"like"')
(1, '0.033*"movie" + 0.010*"one" + 0.009*"like" + 0.008*"good" + 0.007*"great"')
(2, '0.008*"like" + 0.007*"one" + 0.004*"version" + 0.004*"love" + 0.004*"good"')
(3, '0.019*"book" + 0.005*"woman" + 0.004*"time" + 0.003*"information" + 0.003*"would"')
(4, '0.013*"one" + 0.011*"cd" + 0.010*"song" + 0.010*"album" + 0.008*"get"')
